In [2]:
import sys, os, glob, random, copy, time, shutil, re
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
# random.seed(666)
from collections import Counter
import numpy as np
from itertools import cycle
import pandas as pd
import seaborn as sns
from matplotlib import pyplot
import jellyfish
try:
    def hamming_distance(s1, s2):
        if s1 == s2:
            return 0
        else:
            return jellyfish.hamming_distance(s1, s2)
    assert(hamming_distance('ABC', 'ABCD') == 1)
except:
    def hamming_distance(s1, s2):
        if s1 == s2:
            return 0
        else:
            return jellyfish.hamming_distance(unicode(s1), unicode(s2))
    assert(hamming_distance('ABC', 'ABCD') == 1)

import traceback
class nostdout(object):
    def __enter__(self):
        self.stdout = sys.stdout
        sys.stdout = self
    def __exit__(self, type, value, traceback):
        sys.stdout = self.stdout
        if type is not None:
            raise
    def write(self, x): pass

In [3]:
data_dir = 'data2'
f = data_dir+'/SnakeVenom200_aammp_profiles.txt'
table = [l.strip().split(',') for l in open(f)]

In [4]:
headers = table.pop(0)
headers

['clusterID',
 'naiveAA',
 'vdj_len',
 'naive',
 'Nseqs',
 'CDR3_start',
 'CDR3_end',
 'v_grp',
 'd_grp',
 'j_grp',
 'v_gene',
 'd_gene',
 'j_gene',
 'sample',
 'locus',
 'filename',
 'input_seqs',
 'input_seqsAA',
 'Nmuts',
 'abundance',
 'UID',
 'isotype']

In [5]:
headers[10:13] = ['v_allele', 'd_allele', 'j_allele']

In [6]:
df_master = pd.DataFrame(table, columns=headers)

In [7]:
df_master['Nseqs'] = pd.to_numeric(df_master['Nseqs'])
df_master['vdj_len'] = pd.to_numeric(df_master['vdj_len'])
df_master['CDR3_start'] = pd.to_numeric(df_master['CDR3_start'])
df_master['CDR3_end'] = pd.to_numeric(df_master['CDR3_end'])

In [8]:
df_master = df_master.assign(mut_freq=pd.Series(np.array([np.NAN]*len(df_master))).values)
df_master = df_master.assign(major_isotype=pd.Series(np.array([np.NAN]*len(df_master))).values)
df_master = df_master.assign(total_abundance=pd.Series(np.array([np.NAN]*len(df_master))).values)
df_master = df_master.assign(NumbUniqueAA=pd.Series(np.array([np.NAN]*len(df_master))).values)
df_master = df_master.assign(NumbUniqueDNA=pd.Series(np.array([np.NAN]*len(df_master))).values)

a, b, c, d, e = [], [], [], [], []
for i, row in df_master.iterrows():
    if (i+1)%1000==0:
        print(i)
    a.append(np.mean(list(map(int, row['Nmuts'].split(':')))) / (len(row['naiveAA']) * 3))
    b.append(np.sum(list(map(int, row['abundance'].split(':')))))
    c.append(Counter(row['isotype'].split('@')).most_common()[0][0])
    d.append(len(set(row['input_seqsAA'].split(':'))))
    e.append(len(set([s.strip('N') for s in row['input_seqs'].split(':')])))
df_master['mut_freq'] = pd.Series(a)
df_master['total_abundance'] = pd.Series(b)
df_master['major_isotype'] = pd.Series(c)
df_master['NumbUniqueAA'] = pd.Series(d)
df_master['NumbUniqueDNA'] = pd.Series(e)

df_master = df_master.assign(v_gene=pd.Series(np.array([g.split('*')[0] for g in df_master['v_allele']])).values)
df_master = df_master.assign(d_gene=pd.Series(np.array([g.split('*')[0] for g in df_master['d_allele']])).values)
df_master = df_master.assign(j_gene=pd.Series(np.array([g.split('*')[0] for g in df_master['j_allele']])).values)

999
1999
2999
3999
4999
5999
6999
7999
8999
9999
10999
11999
12999
13999
14999
15999
16999
17999
18999
19999
20999
21999
22999
23999
24999
25999
26999
27999
28999
29999
30999
31999
32999
33999
34999
35999
36999
37999
38999
39999
40999
41999
42999
43999
44999
45999
46999
47999
48999
49999
50999
51999
52999
53999
54999
55999
56999
57999
58999
59999
60999
61999
62999
63999
64999
65999
66999
67999
68999
69999
70999
71999
72999
73999
74999
75999
76999
77999
78999
79999
80999
81999
82999
83999
84999
85999
86999
87999
88999
89999
90999
91999
92999
93999
94999
95999
96999
97999
98999
99999
100999
101999
102999
103999
104999
105999


In [9]:
df_master.to_pickle(data_dir+'/cf_master.pickle')

In [10]:
df_master

,clusterID,naiveAA,vdj_len,naive,Nseqs,CDR3_start,CDR3_end,v_grp,d_grp,j_grp,...,UID,isotype,mut_freq,major_isotype,total_abundance,NumbUniqueAA,NumbUniqueDNA,v_gene,d_gene,j_gene
0,0,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,11005,273,306,IGLV1,IGLDx-x*x,IGLJ1,...,AAAAAAAAACGAGGAGG:AAAAAACTGAGCTGAAA:AAAAATAAAA...,IgL:IgL:IgL:IgL:IgL@IgL:IgL:IgL:IgL:IgL:IgL:Ig...,0.018053,IgL,21585,8713,11005,IGLV1,IGLDx-x,IGLJ1
1,1,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,9463,273,306,IGLV1,IGLDx-x*x,IGLJ1,...,AAAAAAAAATATGAGCG:AAAACAGCCAATCACTA:AAAATAAATA...,IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:Ig...,0.018827,IgL,17557,7686,9463,IGLV1,IGLDx-x,IGLJ1
2,2,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,9238,273,306,IGLV1,IGLDx-x*x,IGLJ1,...,AAAAAAAAATAACACCA:CGTCAATCCAATTTAAA@CAGTTATTCG...,IgL:IgL@IgL@IgL@IgL@IgL@IgL@IgL:IgL:IgL:IgL:Ig...,0.018356,IgL,17893,7405,9238,IGLV1,IGLDx-x,IGLJ1
3,3,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,9023,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,AAAAAAAACAACGATTT:AAAAGTCCAATATCAGA:AAACAAAGAC...,IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL@IgL@IgL@Ig...,0.017358,IgL,20441,7074,9023,IGLV2,IGLDx-x,IGLJ2
4,4,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,8733,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,AAAAAAAAGCCAAATCT:AAAAAAGCCTCATTCAA:AAAACAGAGT...,IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL@IgL@Ig...,0.016679,IgL,20868,6791,8733,IGLV2,IGLDx-x,IGLJ2
5,5,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,6179,273,306,IGLV1,IGLDx-x*x,IGLJ1,...,AAAATTTATGCATAACA:AAAATTTTTAATCCACA:AAAATTTTTT...,IgL:IgL:IgL:IgL:IgL@IgL:IgL@IgL:IgL:IgL:IgL@Ig...,0.016998,IgL,11566,5019,6179,IGLV1,IGLDx-x,IGLJ1
6,6,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,5987,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,AAAAAAAAAGCATGGAA:AAAATAGACTTATTTCA:AAACAACCAT...,IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:Ig...,0.019426,IgL,12407,4870,5987,IGLV2,IGLDx-x,IGLJ2
7,7,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,5870,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,AAAAAAAAGTATAGGAC:AAATTTAGACATTCACA:AAGAGTAAAG...,IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:Ig...,0.016539,IgL,13771,4733,5870,IGLV2,IGLDx-x,IGLJ2
8,8,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,5783,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,AAAAAAAATCCTCCATA:AAAAAAAGATGAAACGA:AAAAATATCC...,IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:IgL:Ig...,0.018540,IgL,10802,4787,5783,IGLV2,IGLDx-x,IGLJ2
9,9,DIQMTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQKPDGTVKL...,107,GATATCCAGATGACACAGACTACATCCTCCCTGTCTGCCTCTCTGG...,5764,279,312,IGKV10,IGKDx-x*x,IGKJ1,...,AAAAAAATTATTAAATA:AAAATAGTATGTGTTAG:AAACATATAT...,IgK:IgK:IgK:IgK:IgK:IgK:IgK:IgK@IgK@IgK:IgK:Ig...,0.034939,IgK,13303,4724,5764,IGKV10-96,IGKDx-x,IGKJ1


In [11]:
cl = list()
ex = ['input_seqs', 'input_seqsAA', 'Nmuts', 'abundance', 'isotype']
for i, row in df_master.iterrows():
    if (i+1)%1000==0:
        print(i)
        # break
    a, b, c, d, e = [row[c].split(':') for c in ex]
    e = [Counter(ei.split('@')).most_common()[0][0] for ei in e]
    clIDs = [row['clusterID']] * len(a)
    st, stp = int(row['CDR3_start']), int(row['CDR3_end'])
    CDR3 = [str(Seq(s[st:stp], generic_dna).translate()) for s in a]
    # List append is much faster than dataframe append:
    cl.extend(list(zip(*[clIDs, a,  b, c, d, e, CDR3])))

fdf = pd.DataFrame(cl, columns=['clusterID']+ex+['CDR3'])

999
1999
2999
3999
4999
5999
6999
7999
8999
9999
10999
11999
12999
13999
14999
15999
16999
17999
18999
19999
20999
21999
22999
23999
24999
25999
26999
27999
28999
29999
30999
31999
32999
33999
34999
35999
36999
37999
38999
39999
40999
41999
42999
43999
44999
45999
46999
47999
48999
49999
50999
51999
52999
53999
54999
55999
56999
57999
58999
59999
60999
61999
62999
63999
64999
65999
66999
67999
68999
69999
70999
71999
72999
73999
74999
75999
76999
77999
78999
79999
80999
81999
82999
83999
84999
85999
86999
87999
88999
89999
90999
91999
92999
93999
94999
95999
96999
97999
98999
99999
100999
101999
102999
103999
104999
105999


In [12]:
fdf

,clusterID,input_seqs,input_seqsAA,Nmuts,abundance,isotype,CDR3
0,0,NNNNNNCAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCAC...,QAVVTQESALTTSPGATVTLTCRSSAGAVTTTNNANWVQEKPDHLF...,9,1,IgL,CALWYNNHWVF
1,0,NNNNNNCAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCAC...,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,3,1,IgL,CVLWYSNHWVF
2,0,NNNNNNCAGGCCGTTGTGACTCAGGAATATGCACTCACCACATCAC...,QAVVTQEYALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,2,1,IgL,CALWYSNHWVF
3,0,NNNNNNCAGGCTGTTGTGACTCAGGAATCTGCACTAACCACATCAC...,QAVVTQESALTTSPGETVTLNCRSSTGTVTTSNYANWVQEKPVHLF...,8,1,IgL,CALWYSNHWVF
4,0,NNNNNNCAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCAC...,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,1,2,IgL,CALWYSNHWVF
5,0,NNNNNNCAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCAC...,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,3,1,IgL,CALWYSNHWVF
6,0,NNNNNNCATGCTGTTTTGACTCAGGAATCTGCACTCACCACATCAC...,HAVLTQESALTTSPGETVTLTCRLSTGAVTINNHAKWVQEKPYHLL...,14,1,IgL,CALWYSNHWVF
7,0,NNNNNNCAGGCTGTTGTGACTCCGGAATCTGCACTCACCACATCAC...,QAVVTPESALTTSPGETVTLTCLSSTGAVTTNNYPHWVQDKPDHLF...,10,1,IgL,CALWYSNHWVF
8,0,NNNNNNCAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCAC...,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,3,1,IgL,CALWYSNHWVF
9,0,NNNNNNCATGCTGTTGTGAGTCAGGAATCTGCACTCCCCACATCAC...,HAVVSQESALPTSPGATVTLTCRSSTGAITTSNYANWVQEKPDHLF...,6,1,IgL,CALWYSNHWVF


In [13]:
fdf.to_pickle(data_dir+'/flat_cf.pickle')

In [14]:
len(set(fdf['CDR3']))

181749

In [15]:
df = df_master.copy(deep=True)
df.drop(['input_seqsAA', 'input_seqs', 'UID', 'Nmuts', 'abundance', 'isotype'], inplace=True, axis=1)

In [16]:
df

,clusterID,naiveAA,vdj_len,naive,Nseqs,CDR3_start,CDR3_end,v_grp,d_grp,j_grp,...,locus,filename,mut_freq,major_isotype,total_abundance,NumbUniqueAA,NumbUniqueDNA,v_gene,d_gene,j_gene
0,0,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,11005,273,306,IGLV1,IGLDx-x*x,IGLJ1,...,IgL,PLA2-1_reheader_IgL_split_1_partitions-cluster...,0.018053,IgL,21585,8713,11005,IGLV1,IGLDx-x,IGLJ1
1,1,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,9463,273,306,IGLV1,IGLDx-x*x,IGLJ1,...,IgL,PLA2-2_reheader_IgL_split_1_partitions-cluster...,0.018827,IgL,17557,7686,9463,IGLV1,IGLDx-x,IGLJ1
2,2,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,9238,273,306,IGLV1,IGLDx-x*x,IGLJ1,...,IgL,3FTx-3_reheader_IgL_split_1_partitions-cluster...,0.018356,IgL,17893,7405,9238,IGLV1,IGLDx-x,IGLJ1
3,3,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,9023,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,IgL,PLA2-1_reheader_IgL_split_1_partitions-cluster...,0.017358,IgL,20441,7074,9023,IGLV2,IGLDx-x,IGLJ2
4,4,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,8733,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,IgL,PLA2-2_reheader_IgL_split_1_partitions-cluster...,0.016679,IgL,20868,6791,8733,IGLV2,IGLDx-x,IGLJ2
5,5,QAVVTQESALTTSPGETVTLTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,6179,273,306,IGLV1,IGLDx-x*x,IGLJ1,...,IgL,3FTx-1_reheader_IgL_split_1_partitions-cluster...,0.016998,IgL,11566,5019,6179,IGLV1,IGLDx-x,IGLJ1
6,6,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,5987,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,IgL,3FTx-1_reheader_IgL_split_1_partitions-cluster...,0.019426,IgL,12407,4870,5987,IGLV2,IGLDx-x,IGLJ2
7,7,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,5870,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,IgL,3FTx-3_reheader_IgL_split_1_partitions-cluster...,0.016539,IgL,13771,4733,5870,IGLV2,IGLDx-x,IGLJ2
8,8,QAVVTQESALTTSPGGTVILTCRSSTGAVTTSNYANWVQEKPDHLF...,109,CAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTG...,5783,273,306,IGLV2,IGLDx-x*x,IGLJ2,...,IgL,3FTx-2_reheader_IgL_split_1_partitions-cluster...,0.018540,IgL,10802,4787,5783,IGLV2,IGLDx-x,IGLJ2
9,9,DIQMTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQKPDGTVKL...,107,GATATCCAGATGACACAGACTACATCCTCCCTGTCTGCCTCTCTGG...,5764,279,312,IGKV10,IGKDx-x*x,IGKJ1,...,IgK,PLA2-1_reheader_IgK_split_1_partitions-cluster...,0.034939,IgK,13303,4724,5764,IGKV10-96,IGKDx-x,IGKJ1


In [17]:
df.to_pickle(data_dir+'/cf.pickle')